5주차 예습과제에서는 6주차 내용인 GPT의 생성 능력을 맛보고자 합니다. 사용할 모델은 koalpaca라는 모델인데, 한국어 버전의 alpaca 모델입니다.

alpaca는 llama라는 모델을 instruction fine-tuning 해서 사람의 지시를 잘 따르는 모델입니다.

llama라는 모델은 LLaMA-13B의 경우 GPT-3보다 10배이상 작음에도 불구하고 대부분의 평가서 GPT-3보다 우수한 성능을 보이며, 더 나아가 LLaMA-65B의 경우 대부분의 벤치마크에서 Chinchilla, Gopher, GPT-3, PaLM와 유사하거나 더 뛰어난 결과를 보이는 훌륭한 모델입니다.

그러면 지금부터 koalpaca로 생성 모델을 한번 경험해보도록 하겠습니다!

In [1]:
# GPU 켜져 있는지 확인

!nvidia-smi

Wed Aug  7 08:24:54 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

peft는 parameter efficient fine-tuning의 약자로, 큰 모델을 코랩에서 돌릴 수 있도록 쪼개서 만든 것입니다.

오래 걸립니다!

In [4]:
from peft import PeftModel, PeftConfig

In [5]:
peft_model_id = "beomi/qlora-koalpaca-polyglot-12.8b-50step"
config = PeftConfig.from_pretrained(peft_model_id)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config, device_map={"":0})
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/428 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/52.5k [00:00<?, ?B/s]

model-00001-of-00028.safetensors:   0%|          | 0.00/946M [00:00<?, ?B/s]

...

adapter_model.bin:   0%|          | 0.00/26.2M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 5120)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-39): 40 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXSdpaAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=5120, out_features=15360, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
            

next token generation

In [6]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 '문제들'을 해결함으로써 인류의 삶을 진화시킬 것으로 예상되고 있습니다. 최근 '이세돌 9단'과의 대국으로 더욱 유명해져 향후 구글, 애플 등이


In [10]:
# 입력 프롬프트
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'

# 입력을 토큰화
tokens = tokenizer.encode(prompt, return_tensors='pt').to('cuda', non_blocking=True)

# attention_mask 추가
attention_mask = tokens.ne(tokenizer.pad_token_id).long().to('cuda', non_blocking=True)

# 텍스트 생성
with torch.no_grad():
    gen_tokens = model.generate(tokens, attention_mask=attention_mask, do_sample=True, temperature=0.8, max_length=64)
    generated = tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)[0]

print(generated)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 난제들을 해결할 수 있으리라 기대한다. 그는 뇌과학을 통해 인간의 마음을 읽는 것에 그치는 것이 아니라, 그 마음을 읽음으로써 오히려 인류의 난제를


 QA

In [11]:
def gen(x):
    q = f"### 질문: {x}\n\n### 답변:"
    # print(q)
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        max_new_tokens=50,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [12]:
gen('건강하게 살기 위한 세 가지 방법은?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 건강하게 살기 위한 세 가지 방법은?

### 답변: (3) 규칙적인 생활과 정기검진(혈액검사,X-ray,내시경검사)으로 자신을 체크해 나가자. - 정기적인 검사로 질병이 생기기 전에 막자. - 좋은 생활


top_k와 같은 parameter를 조절하면서 실험해보세요!

In [13]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64, top_k=50)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 난제들을 해결해 왔다. ​​한편, 인공지능이 갖고 있는 잠재력이 인류에게 위협적인 요소로 작용할 수도 있다는 우려도 있다. 인간처럼 생각


In [14]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=500, top_k=50)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 문제를 해결할 수 있는 열쇠가 될 수 있다.​또한, 인공지능은 인간을 '심적으로' 만족시킬 수 있어 인간의 심리적 안정감과 행복감을 제공할 수 있다. 과거보다 훨씬 더 많은 인간의 고민을 해결할 수 있는 기술이 되는 것이다. 이에 대해 필자는 '인간을 위한 인공 지능'이라는 표현을 사용했다.​인간과 소통할 수 있는 인공 지능, 감성을 자극하는 인공 지능이 미래의 인공 지능에서 주목해야 할 핵심 포인트다.​ 2. 인공 지능의 발전 방향과 기대되는 미래​미래에 인공 지능이 얼마나 발전할 지, 어떤 방향으로 발전할 지 알기 위해서는 '인공 지능'의 발전 과정을 알아봐야 한다.​인공 지능은 기본적으로 아래와 같이 발전해왔다.​1. 인공 지능의 시초, 기호 학습​인공 지능이라는 단어가 생기기 이전부터 인간은 '기호 학습'이라는 방법을 이용해 인공 지능을 연구해왔다.​기호 학습은 '인공 지능의 아버지'라고 불리는 앨런 튜링이 발견한 것으로, 기계가 인간처럼 지능을 갖기 위해서는 인간의 뇌와 비슷하게 만들어야 한다는 아이디어에서 시작된 것이다.​인간의 지능은 기본적으로 '기호'인 언어를 이해하고, 그 언어를 이용해 '생각'을 하고 '판단'을 내리기 때문에 인공 지능도 언어를 이해하고 그 언어를 이용해 생각하고 판단하는 것을 연구한 것이다.​1950년대 후반, '기호 학습'이라는 것이 발표되면서 인공 지능 분야 연구에 불이 붙기 시작했다.​이 기호 학습에는 '기호'를 학습하고 나면, 해당 기호에 대한 '개념'을 익혀야 한다는 한계가 있다.​2. '개념'을 배우는 인공 지능​이렇게 기호만을 학습하는 인공 지능에서, '개념'을 배울 수 있는 인공 지능으로 발전하기 시작했다.​언어를 이해하는 능력을 넘어서, 언어로 소통할 수 있는 새로운 언어, 즉 '개념'도 배울 수 있는 인공 지능으로 발전한 것


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

문제가 발생
-> 결과: 문장의 완결이 안됨

1. attention mask 설정
2. pad toekn id 설정
이 필요함.

pad token id를 eos_token_id로 설정해주겠음


In [15]:
# 입력 프롬프트
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'

# 입력을 토큰화
tokens = tokenizer.encode(prompt, return_tensors='pt').to('cuda', non_blocking=True)
#입력 프롬프트를 토큰화하여 PyTorch 텐서(pt) 형식으로 변환

# attention_mask 추가
attention_mask = tokens.ne(tokenizer.pad_token_id).long().to('cuda', non_blocking=True)
#tokens.ne(tokenizer.pad_token_id): 토큰화된 텐서에서 패딩 토큰(pad_token_id)과 일치하지 않는 위치를 찾음
#ne는 "not equal"을 의미

# 텍스트 생성
with torch.no_grad():
    gen_tokens = model.generate(tokens, attention_mask=attention_mask, do_sample=True, temperature=0.4, max_length=600)
    #Top-K 샘플링: 모델이 다음 단어를 생성할 때, 예측된 확률 분포에서 상위 K개의 확률이 높은 단어들만 고려하고, 나머지 단어들은 무시-> 상위 K개의 단어들 중에서 하나를 샘플링하여 다음 단어로 선택
    #-> 무작위성을 도입하면서도 가장 가능성이 높은 단어들만 고려하여 생성된 텍스트의 품질을 유지하는 데 도움
    #top_k 값을 낮추면 텍스트가 더 예측 가능해지고, 높이면 더 창의적이게 됨
    #do_sample=True: 샘플링을 사용하여 텍스트를 생성-> 더 다양하고 예측 불가능한 출력
    #temperature=0.4: 샘플링의 확률 분포를 조절-> 낮은 값은 더 예측 가능한 출력을, 높은 값은 더 창의적인 출력을 생성
    generated = tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)[0]
    #생성된 토큰 시퀀스를 텍스트로 변환
    #skip_special_tokens=True: 특별 토큰(예: PAD, BOS, EOS 등)을 제거

print(generated)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 난제들을 해결할 수 있을 것으로 기대된다. ​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​


top_k=50을 추가하자 문장이 길어짐?

In [16]:
# 입력 프롬프트
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'

# 입력을 토큰화
tokens = tokenizer.encode(prompt, return_tensors='pt').to('cuda', non_blocking=True)
#입력 프롬프트를 토큰화하여 PyTorch 텐서(pt) 형식으로 변환

# attention_mask 추가
attention_mask = tokens.ne(tokenizer.pad_token_id).long().to('cuda', non_blocking=True)
#tokens.ne(tokenizer.pad_token_id): 토큰화된 텐서에서 패딩 토큰(pad_token_id)과 일치하지 않는 위치를 찾음
#ne는 "not equal"을 의미

# 텍스트 생성
with torch.no_grad():
    gen_tokens = model.generate(tokens, attention_mask=attention_mask, do_sample=True, temperature=0.4, max_length=600,top_k=50)
    generated = tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)[0]

print(generated)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 문제들을 해결해 나가는 것이다.​AI는 이미 우리 생활 속에 깊숙히 들어와 있다. 스마트폰의 음성인식 기능, 자율주행 자동차, 인공지능 스피커, 챗봇, 드론 택배, 로봇 청소기, 스마트 TV, 인공지능 비서, 스마트 냉장고, 스마트 세탁기, 스마트 에어컨, 스마트 공기청정기, 스마트 조명, 스마트 TV, 스마트 홈, 스마트 헬스케어, 스마트 팜, 스마트 시티, 스마트 팩토리, 스마트 카, 스마트 가전, 스마트 홈, 스마트 헬스케어, 스마트 팜, 스마트 시티, 스마트 팩토리, 스마트 카, 스마트 가전, 스마트 홈 등....​지능형 제품과 서비스에 대한 수요가 폭발적으로 증가하고 있다.​AI는 생활의 편리함을 넘어 인간의 삶을 더욱 풍요롭게 해주는 방향으로 발전하고 있다.​AI는 인간의 삶을 편리하게 해주는 것을 넘어 인간의 삶을 풍요롭게 해주는 방향으로 발전하고 있다.​AI는 인간의 삶을 편리하게 해주는 것을 넘어 인간의 삶을 풍요롭게 해주는 방향으로 발전하고 있다.​AI는 인간의 삶을 편리하게 해주는 것을 넘어 인간의 삶을 풍요롭게 해주는 방향으로 발전하고 있다.​AI는 인간의 삶을 편리하게 해주는 것을 넘어 인간의 삶을 풍요롭게 해주는 방향으로 발전하고 있다.​AI는 인간의 삶을 편리하게 해주는 것을 넘어 인간의 삶을 풍요롭게 해주는 방향으로 발전하고 있다.​AI는 인간의 삶을 편리하게 해주는 것을 넘어 인간의 삶을 풍요롭게 해주는 방향으로 발전하고 있다.​AI는 인간의 삶을 편리하게 해주는 것을 넘어 인간의 삶을 풍요롭게 해주는 방향으로 발전하고 있다.​AI는 인간의 삶을 편리하게 해주는 것을 넘어 인간의 삶을 풍요롭게 해주는 방향으로 발전하고 있다.​AI는 인간의 삶을 편리하게 해주는 것을 넘어 인간의 삶을 풍요롭게 해주는 방향으로 발전하고 있다.​AI는 인간의 삶을 편리하게 해주는 것을 넘어 인간의 삶을 풍요롭게 해주는 방향으로 발전하고 있다.​AI는 인간의 삶을 편리하게 해주는 

temperature를 올리고
top_k를 낮추니까
의문형의 문장을 생성

+ 참고자료를 출력하기 시작함.


In [17]:
# 입력 프롬프트
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'

# 입력을 토큰화
tokens = tokenizer.encode(prompt, return_tensors='pt').to('cuda', non_blocking=True)
#입력 프롬프트를 토큰화하여 PyTorch 텐서(pt) 형식으로 변환

# attention_mask 추가
attention_mask = tokens.ne(tokenizer.pad_token_id).long().to('cuda', non_blocking=True)
#tokens.ne(tokenizer.pad_token_id): 토큰화된 텐서에서 패딩 토큰(pad_token_id)과 일치하지 않는 위치를 찾음
#ne는 "not equal"을 의미

# 텍스트 생성
with torch.no_grad():
    gen_tokens = model.generate(tokens, attention_mask=attention_mask, do_sample=True, temperature=1.0, max_length=600,top_k=20)
    generated = tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)[0]

print(generated)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 문제들을 해결할 수 있다는 주장에 대해 어떻게 생각하시나요?- 인공지능은 어디까지 개발될 것으로 생각하나요? 그리고 인간과 비교하여 어느 정도의 수준에 이를 거라고 생각하세요?- AI가 발전한다면 어떤 영역에서 더 활용될 것으로 보시나요?- 인공지능은 사회와 인간을 어떻게 바꿀 것이라고 생각하시나요?- 현재 인공지능에 대한 연구는 왜 더딘가요?- 인공지능을 통해 우리 사회가 더 편리해질 수 있을까요?- 인공지능 기술의 활용은 사람들의 일과 삶에 어떤 영향을 미칠 것이라 생각하시나요? ​- 인공지능이 발달하면 미래에 사라질 직업에 대해서도 관심이 많은데요. 그에 대한 이야기를 나눠 보세요. ​​​​​​​​​​​​​[참고 링크] ​- https://news.joins.com/article/16313736​- http://www.zdnet.co.kr/view/?no=20190408175059 ​- https://www.hankyung.com/news/article/2017122389011​- https://blog.naver.com/wjdgk0129/221502332487 ​- http://naver.me/55vhUjgT​- https://mk.co.kr/news/view.php?year=2010&no=890913​- http://www.ddaily.co.kr/news/article/?no=137468​- https://mnews.joins.com/amparticle/24061233?cloc=joongang%7Cmhome%7Ctopnews​- https://mnews.joins.com/amparticle/23950508#home​- http://news.mk.co.kr/v2/economy/view.php?year=2010&no=890913​- https://mk.co.kr/news/view.php?year=2010&
